In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os
import matplotlib.pyplot as plt
import random
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# TODO:
# - More users
# - Different n gram ns
# - SVM
# - Word

random.seed(0)

# https://mjeensung.github.io/characterbigramtfidf/

# Load data
drive.mount('/content/gdrive', force_remount=True)
data_dir = '../content/gdrive/MyDrive/544/data'

# Load dataset
with open(os.path.join(data_dir, "X_train.npy"), 'rb') as f:
    X_train = np.load(f)
with open(os.path.join(data_dir, "X_val.npy"), 'rb') as f:
    X_val = np.load(f)
with open(os.path.join(data_dir, "X_test.npy"), 'rb') as f:
    X_test = np.load(f)
with open(os.path.join(data_dir, "X_other.npy"), 'rb') as f:
    X_other = np.load(f)


"""from sklearn.random_projection import SparseRandomProjection
def project(X):
  transformer = SparseRandomProjection(n_components=1000, eps=0.2)
  return transformer.fit_transform(X)"""

# def project(X):
#   return X / np.linalg.norm(X,axis=1).reshape((-1, 1))

X_train = X_train
X_val = X_val
X_test = X_test
X_other = X_other

y_train = pd.read_csv(os.path.join(data_dir, "y_train.csv"))
y_val = pd.read_csv(os.path.join(data_dir, "y_val.csv"))
y_test = pd.read_csv(os.path.join(data_dir, "y_test.csv"))

Mounted at /content/gdrive


In [2]:
X_other.shape

(3157, 7500)

In [3]:
"""lda = LinearDiscriminantAnalysis(n_components=20)
lda.fit(X_train, y_train)

X_train = lda.transform(X_train)
X_val = lda.transform(X_val)
X_test = lda.transform(X_test)
X_other = lda.transform(X_other)"""

'lda = LinearDiscriminantAnalysis(n_components=20)\nlda.fit(X_train, y_train)\n\nX_train = lda.transform(X_train)\nX_val = lda.transform(X_val)\nX_test = lda.transform(X_test)\nX_other = lda.transform(X_other)'

In [4]:
X_val.shape

(1750, 7500)

In [5]:
labels = np.unique(np.concatenate([y_train, y_test, y_val]))

In [6]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

def get_user_thresh(user_label, X_train, y_train):
  is_user = y_train["user"] == user_label   
  X_user = X_train[is_user.squeeze()]   
  neigh = NearestNeighbors(n_neighbors=5, n_jobs=-1)
  neigh.fit(X_user)
  dist, ind = neigh.kneighbors(X_user)
  thresh = np.percentile(dist, 25)
  return thresh

def get_accuracy(final_preds, y_pred):
  y_pred = y_pred["user"].values
  return np.mean([p[0] == l for p, l in zip(final_preds, y_pred)])

def predict(neigh, X_pred, y_train):
  # y_pred = y_pred["user"].values
  y_train_vals = y_train["user"].values

  dist, ind = neigh.kneighbors(X_pred)
  pred_labels = [[y_train_vals[i] for i in pred_idx] for pred_idx in ind]
  thresh_pred = []

  modes = pd.DataFrame(pred_labels).mode(axis=1)

  mode_dists = []
  for i_row in range(len(modes)):
    row_mode = modes.iloc[i_row].values
    row_preds = pred_labels[i_row]
    row_dists = dist[i_row]
    thresh_pred.append(user_thresholds[row_mode[0]])

    row_min_dists = {}
    for m in row_mode:
      if not isinstance(m, str) and np.isnan(m):
        break
        
      label_dists = [d for d, pred in zip(row_dists, row_preds) if pred == m]
      #print(label_dists)
      row_min_dists[m] = np.min(label_dists)

    mode_dists.append(row_min_dists)

  # Get the prediction and dist for each example
  min_dists = []
  for d in mode_dists:
    i_min = np.argmin(list(d.values()))
    min_key = list(d.keys())[i_min]
    min_dists.append((min_key, d[min_key]))

  # Add reject/accept
  final_preds = [(p[0], p[1], p[1] <= user_th) for p, user_th in zip(min_dists, thresh_pred)]
  return final_preds

user_thresholds = {l: get_user_thresh(l, X_train, y_train) for l in tqdm(labels)}

100%|██████████| 50/50 [00:02<00:00, 16.73it/s]


In [7]:
neigh = NearestNeighbors(n_neighbors=3, n_jobs=-1)
neigh.fit(X_train)

X_pred = X_train
#y_pred = neigh.kneighbors(X_train)
train_preds = predict(neigh, X_pred, y_train)
train_acc = get_accuracy(train_preds, y_train)
print(f"Training accuracy: {train_acc:.3f}")

Training accuracy: 0.959


# Rejection performance

In [8]:
other_preds = predict(neigh, X_other, y_train)
rejection_accuracy = 1 - np.mean([p[2] for p in other_preds])
print(f"Rejection accuracy: {rejection_accuracy:.3f}")

Rejection accuracy: 0.497


# Validation set

In [9]:
X_pred = X_val
val_preds = predict(neigh, X_pred, y_train)
val_acc = get_accuracy(val_preds, y_val)
print(f"Validation accuracy: {val_acc:.3f}")

Validation accuracy: 0.190


In [10]:
val_preds = [p[0] for p in val_preds]
y_val = y_val["user"].values
val_f1 = metrics.f1_score(y_val, val_preds, average = 'macro')
print(f"F1 score on val set: {val_f1:.3f}")

F1 score on val set: 0.192


# Test set

In [11]:
X_pred = X_test
test_preds = predict(neigh, X_pred, y_train)
test_acc = get_accuracy(test_preds, y_test)
print(f"Test accuracy: {test_acc}")

test_preds = [p[0] for p in test_preds]
# print(classification_report(y_test["user"].values, test_preds))
test_f1 = metrics.f1_score(y_true = y_test["user"].values, y_pred = test_preds, average = 'weighted')
print(f"F1 score on test set: {test_f1:.3f}")

Test accuracy: 0.179
F1 score on test set: 0.180
